In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Pneumothorax"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
522: 3705
32: 3705
711: 3705
662: 3705
291: 3705
232: 3705
23: 3705
3705: 3705
173: 3705
24: 3705
539: 3705
62: 3705
192: 3705
7: 3705
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
3693/3705 [============================>.] - ETA: 0s - loss: 4.6033 - leftLayer1_loss: 0.0876 - midLayer1_loss: 1.3394 - rightLayer1_loss: 0.8681 - leftLayer2_loss: 0.0564 - midLayer2_loss: 1.4802 - rightLayer2_loss: 0.7717
Epoch 00006: val_loss improved from 4.74651 to 4.67896, saving model to ./experiments/0.009_weights.h5
3705/3705 [==============================] - 10s 3ms/step - loss: 4.6037 - leftLayer1_loss: 0.0876 - midLayer1_loss: 1.3394 - rightLayer1_loss: 0.8682 - leftLayer2_loss: 0.0563 - midLayer2_loss: 1.4803 - rightLayer2_loss: 0.7718 - val_loss: 4.6790 - val_leftLayer1_loss: 0.0863 - val_midLayer1_loss: 1.3429 - val_rightLayer1_loss: 0.8705 - val_leftLayer2_loss: 0.0762 - val_midLayer2_loss: 1.3607 - val_rightLayer2_loss: 0.9423
Epoch 7/11
3690/3705 [============================>.] - ETA: 0s - loss: 4.5639 - leftLayer1_loss: 0.0825 - midLayer1_loss: 1.3392 - rightLayer1_loss: 0.8424 - leftLayer2_loss: 0.0521 - midLayer2_loss: 1.4847 - rightLayer2_loss: 0.7629

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 3705 steps, validate for 504 steps
Epoch 1/11
3701/3705 [============================>.] - ETA: 0s - loss: 6.2111 - leftLayer1_loss: 0.1167 - midLayer1_loss: 1.4155 - rightLayer1_loss: 1.6141 - leftLayer2_loss: 0.1159 - midLayer2_loss: 1.6046 - rightLayer2_loss: 1.3444
Epoch 00001: val_loss improved from inf to 5.73546, saving model to ./experiments/0.009999999999999998_weights.h5
3705/3705 [==============================] - 10s 3ms/step - loss: 6.2105 - leftLayer1_loss: 0.1167 - midLayer1_loss: 1.4155 - rightLayer1_loss: 1.6138 - leftLayer2_loss: 0.1159 - midLayer2_loss: 1.6045 - rightLayer2_loss: 1.3441 - val_loss: 5.7355 - val_leftLayer1_loss: 0.1119 - val_midLayer1_loss: 1.4058 - val_rightLayer1_loss: 1.3985 - val_leftLayer2_loss: 0.1132 - val_midLayer2_loss: 1.4440 - val_rightLayer2_loss: 1.

3697/3705 [============================>.] - ETA: 0s - loss: 4.6521 - leftLayer1_loss: 0.0626 - midLayer1_loss: 1.4159 - rightLayer1_loss: 0.7819 - leftLayer2_loss: 0.0414 - midLayer2_loss: 1.6046 - rightLayer2_loss: 0.7456
Epoch 00011: val_loss improved from 4.61604 to 4.59612, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Pneumothorax: 0.6371718367744501

auprc 0Pneumothorax: 0.07350588334798666

auroc 1Pneumothorax: 0.3206415043167121

auprc 1Pneumothorax: 0.0323283309702635

auroc 2Pneumothorax: 0.6578750268460811

auprc 2Pneumothorax: 0.09123116914726159

auroc 3Pneumothorax: 0.4448730352454626

auprc 3Pneumothorax: 0.04176844170334609

auroc 4Pneumothorax: 0.5410104219584423

auprc 4Pneumothorax: 0.05874833405848391

auroc 5Pneumothorax: 0.6459462460574121

auprc 5Pneumothorax: 0.08208535505811582

mean auroc: 0.5412530118664267

mean a

Epoch 4/11
3689/3705 [============================>.] - ETA: 0s - loss: 4.8875 - leftLayer1_loss: 0.0980 - midLayer1_loss: 1.4250 - rightLayer1_loss: 0.9536 - leftLayer2_loss: 0.0713 - midLayer2_loss: 1.5347 - rightLayer2_loss: 0.8049
Epoch 00004: val_loss improved from 5.02711 to 4.88729, saving model to ./experiments/0.010999999999999998_weights.h5
3705/3705 [==============================] - 10s 3ms/step - loss: 4.8879 - leftLayer1_loss: 0.0980 - midLayer1_loss: 1.4250 - rightLayer1_loss: 0.9537 - leftLayer2_loss: 0.0712 - midLayer2_loss: 1.5348 - rightLayer2_loss: 0.8051 - val_loss: 4.8873 - val_leftLayer1_loss: 0.0951 - val_midLayer1_loss: 1.4129 - val_rightLayer1_loss: 0.9325 - val_leftLayer2_loss: 0.0845 - val_midLayer2_loss: 1.3882 - val_rightLayer2_loss: 0.9742
Epoch 5/11
3703/3705 [============================>.] - ETA: 0s - loss: 4.7979 - leftLayer1_loss: 0.0915 - midLayer1_loss: 1.4250 - rightLayer1_loss: 0.8969 - leftLayer2_loss: 0.0631 - midLayer2_loss: 1.5378 - rightLaye

...

3690/3705 [============================>.] - ETA: 0s - loss: 4.4533 - leftLayer1_loss: 0.0607 - midLayer1_loss: 1.3972 - rightLayer1_loss: 0.7750 - leftLayer2_loss: 0.0409 - midLayer2_loss: 1.4363 - rightLayer2_loss: 0.7433
Epoch 00011: val_loss improved from 4.50767 to 4.48823, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Pneumothorax: 0.5231419457282378

auprc 0Pneumothorax: 0.05094752142526212

auroc 1Pneumothorax: 0.4379845846704747

auprc 1Pneumothorax: 0.0415674441136443

auroc 2Pneumothorax: 0.6110547085272791

auprc 2Pneumothorax: 0.06613858553416234

auroc 3Pneumothorax: 0.4170196237969169

auprc 3Pneumothorax: 0.03959424977989118

auroc 4Pneumothorax: 0.3619898039960736

auprc 4Pneumothorax: 0.03425343772710811

auroc 5Pneumothorax: 0.5285480968193589

auprc 5Pneumothorax: 0.05142194928865085

mean auroc: 0.4799564605897235

mean a

Epoch 4/11
3687/3705 [============================>.] - ETA: 0s - loss: 4.6214 - leftLayer1_loss: 0.0945 - midLayer1_loss: 1.3166 - rightLayer1_loss: 0.9180 - leftLayer2_loss: 0.0706 - midLayer2_loss: 1.4165 - rightLayer2_loss: 0.8051
Epoch 00004: val_loss improved from 4.84673 to 4.71847, saving model to ./experiments/0.012999999999999996_weights.h5
3705/3705 [==============================] - 10s 3ms/step - loss: 4.6218 - leftLayer1_loss: 0.0945 - midLayer1_loss: 1.3167 - rightLayer1_loss: 0.9181 - leftLayer2_loss: 0.0706 - midLayer2_loss: 1.4167 - rightLayer2_loss: 0.8052 - val_loss: 4.7185 - val_leftLayer1_loss: 0.0915 - val_midLayer1_loss: 1.3175 - val_rightLayer1_loss: 0.8989 - val_leftLayer2_loss: 0.0858 - val_midLayer2_loss: 1.3476 - val_rightLayer2_loss: 0.9772
Epoch 5/11
3695/3705 [============================>.] - ETA: 0s - loss: 4.5399 - leftLayer1_loss: 0.0875 - midLayer1_loss: 1.3164 - rightLayer1_loss: 0.8692 - leftLayer2_loss: 0.0626 - midLayer2_loss: 1.4191 - rightLaye

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 3705 steps, validate for 504 steps
Epoch 1/11
3689/3705 [============================>.] - ETA: 0s - loss: 6.1104 - leftLayer1_loss: 0.1157 - midLayer1_loss: 1.4580 - rightLayer1_loss: 1.6362 - leftLayer2_loss: 0.1167 - midLayer2_loss: 1.4675 - rightLayer2_loss: 1.3164
Epoch 00001: val_loss improved from inf to 5.68133, saving model to ./experiments/0.013999999999999995_weights.h5
3705/3705 [==============================] - 10s 3ms/step - loss: 6.1085 - leftLayer1_loss: 0.1157 - midLayer1_loss: 1.4582 - rightLayer1_loss: 1.6352 - leftLayer2_loss: 0.1167 - midLayer2_loss: 1.4676 - rightLayer2_loss: 1.3152 - val_loss: 5.6813 - val_leftLayer1_loss: 0.1108 - val_midLayer1_loss: 1.4538 - val_rightLayer1_loss: 1.4075 - val_leftLayer2_loss: 0.1092 - val_midLayer2_loss: 1.3690 - val_rightLayer2_loss: 1.

3695/3705 [============================>.] - ETA: 0s - loss: 4.5472 - leftLayer1_loss: 0.0612 - midLayer1_loss: 1.4584 - rightLayer1_loss: 0.7796 - leftLayer2_loss: 0.0407 - midLayer2_loss: 1.4663 - rightLayer2_loss: 0.7409
Epoch 00011: val_loss improved from 4.57108 to 4.55209, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Pneumothorax: 0.4335276404497476

auprc 0Pneumothorax: 0.04068765044301817

auroc 1Pneumothorax: 0.7771055071637736

auprc 1Pneumothorax: 0.13944290310417207

auroc 2Pneumothorax: 0.5753074306510657

auprc 2Pneumothorax: 0.06466515939962447

auroc 3Pneumothorax: 0.5307192521163661

auprc 3Pneumothorax: 0.052385143983576354

auroc 4Pneumothorax: 0.39797697569947815

auprc 4Pneumothorax: 0.038519554039830504

auroc 5Pneumothorax: 0.48579964924562513

auprc 5Pneumothorax: 0.048163119186425074

mean auroc: 0.533406075887676

m

Epoch 4/11
3684/3705 [============================>.] - ETA: 0s - loss: 4.8008 - leftLayer1_loss: 0.0950 - midLayer1_loss: 1.3522 - rightLayer1_loss: 0.9799 - leftLayer2_loss: 0.0634 - midLayer2_loss: 1.5212 - rightLayer2_loss: 0.7889
Epoch 00004: val_loss improved from 4.92342 to 4.78792, saving model to ./experiments/0.014999999999999994_weights.h5
3705/3705 [==============================] - 10s 3ms/step - loss: 4.8007 - leftLayer1_loss: 0.0950 - midLayer1_loss: 1.3523 - rightLayer1_loss: 0.9798 - leftLayer2_loss: 0.0634 - midLayer2_loss: 1.5213 - rightLayer2_loss: 0.7889 - val_loss: 4.7879 - val_leftLayer1_loss: 0.0934 - val_midLayer1_loss: 1.3524 - val_rightLayer1_loss: 0.9565 - val_leftLayer2_loss: 0.0750 - val_midLayer2_loss: 1.3990 - val_rightLayer2_loss: 0.9116
Epoch 5/11
3694/3705 [============================>.] - ETA: 0s - loss: 4.7080 - leftLayer1_loss: 0.0895 - midLayer1_loss: 1.3521 - rightLayer1_loss: 0.9171 - leftLayer2_loss: 0.0562 - midLayer2_loss: 1.5206 - rightLaye

In [17]:
print(np.max(maxi))

0.7771055071637736
